<a href="https://colab.research.google.com/github/Amritha07dec/MCN/blob/main/Neural_Network_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Step 1: Padding function
def pad_samples(raw_samples, target_dim=6):
    padded = []
    for ts in raw_samples:
        pad_width = target_dim - ts.shape[1]
        if pad_width > 0:
            ts = np.pad(ts, ((0, 0), (0, pad_width)), mode='constant')
        padded.append(ts)
    return np.array(padded)

# Step 2: Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = torch.tensor(samples, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]

# Step 3: LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=6, hidden_size=64, num_layers=1, num_classes=4):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1])
        return out

# Step 4: Training loop
def train(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

        acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f} | Accuracy: {acc:.2f}%")


In [2]:
# 5. Simulate dummy data
np.random.seed(42)
torch.manual_seed(42)

raw_samples = [np.random.rand(10000, np.random.choice([2, 3, 4, 5, 6])) for _ in range(450)]
labels = np.random.randint(0, 4, size=(450,))

X = pad_samples(raw_samples, target_dim=6)

dataset = TimeSeriesDataset(X, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

model = LSTMClassifier(input_size=6)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train(model, dataloader, criterion, optimizer, epochs=5)

Epoch 1/5 | Loss: 40.0959 | Accuracy: 27.33%
Epoch 2/5 | Loss: 40.0703 | Accuracy: 28.67%
Epoch 3/5 | Loss: 40.1133 | Accuracy: 27.56%
Epoch 4/5 | Loss: 40.1190 | Accuracy: 28.00%
Epoch 5/5 | Loss: 40.0875 | Accuracy: 28.89%


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [33]:
import os
import pickle

# Replace this with your actual folder path inside Google Drive
folder_path = '/content/unzipped_ode_simulations'


In [28]:
import os
print(os.listdir(folder_path))


['ode_simulations.zip']


In [29]:
import zipfile

zip_path = '/content/drive/MyDrive/sample_generation/ode_simulations.zip'
extract_folder = '/content/unzipped_ode_simulations'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)


In [42]:
time_series_list = []
labels = []


"""is it necessary for the neural network to have data  and labels in lists"""


for filename in os.listdir(folder_path):
    if filename.endswith('.pkl') or filename.endswith('.pickle'):
        print(f"Loading file: {filename}")  # 👈 This line shows the filename
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
            sol = data["Time series"]  # shape: (10000, num_features)
            degree = data["degree"]    # class label: 0, 1, 2, or 3

            time_series_list.append(sol)
            labels.append(degree)

print(f"Loaded {len(time_series_list)} samples.")
print(f"First sample shape: {time_series_list.shape}")
#print(f"First label: {labels[0]}")



Loading file: Rossler_Set-3_Deg-2_Params-0.2_0.2_10.0_IC-2.5_0.0_0.0.pkl
Loading file: Lotka_Volterra_Cubic_Set-1_Deg-3_Params-1.0_0.5_0.1_1.0_0.1_IC-0.0_0.0.pkl
Loading file: Lotka_Volterra_Cubic_Set-1_Deg-3_Params-1.0_0.5_0.1_1.0_0.1_IC-1.125_2.25.pkl
Loading file: Lorenz_Set-1_Deg-2_Params-10.0_28.0_2.6666666666666665_IC-0.25_0.25_0.25.pkl
Loading file: Linear_2D_Harmonic_Oscillator_Set-2_Deg-1_Params-0.25_IC-0.5_0.0.pkl
Loading file: Linear_5D_Coupled_Oscillators_Set-1_Deg-1_Params-1.0_1.0_1.0_1.0_1.0_IC-1.5_0.0_0.0_0.0_0.0.pkl
Loading file: Linear_2D_Harmonic_Oscillator_Set-1_Deg-1_Params-1.0_IC-2.0_0.0.pkl
Loading file: Linear_2D_Harmonic_Oscillator_Set-1_Deg-1_Params-1.0_IC-1.0_0.0.pkl
Loading file: Lorenz96_Set-1_Deg-2_Params-10.0_4_IC-0.0_1.25_2.5_3.75.pkl
Loading file: Lorenz_Set-1_Deg-2_Params-10.0_28.0_2.6666666666666665_IC-0.5_0.5_0.5.pkl
Loading file: Linear_1D_Set-2_Deg-1_Params--0.5_IC-2.0.pkl
Loading file: Linear_1D_Set-1_Deg-1_Params-0.5_IC--0.25.pkl
Loading file: Lin

AttributeError: 'list' object has no attribute 'shape'

In [40]:
labels[0:5]

[2, 3, 3, 2, 1]

Loaded 0 samples.


[]